# ShortRead

## Setup

In [ ]:
library(ShortRead)
library(tidyverse)
# library(Biostrings)
set.seed(100)

In [ ]:
### Specify FASTQ file
myfqdir <- "/data/hts2018_pilot/Granek_4837_180427A5/"
myfqname <- "35_MA_P_S39_L002_R1_001.fastq.gz"
myfqfile <- file.path(myfqdir, myfqname)
outdir = path.expand("~/work/scratch/shortread")
dir.create(outdir)
tools::md5sum(myfqfile)

## FASTQ Basics

In [ ]:
### Read entire FASTQ file into memory
myfq <- readFastq(myfqfile)

### Check class
class(myfq)

In [ ]:
### Get Summary for FASTQ file
myfq

In [ ]:
### Get number of reads
length(myfq)

In [ ]:
### Subset based on first three records
myfq[1:3]

In [ ]:
### Subset based on three randomly selected records
myfq[sample(seq_len(length(myfq)),3, replace=FALSE)]

In [ ]:
### Get sequences and phred scores for reads 1,2, 3 and 1 (add duplicate on
### purpose
myrecs <- myfq[c(1,2,3,1)]

sread(myrecs) -> myseqs

quality(myrecs) -> myphreds

### Compare classes
class(myfq)
class(myseqs)
class(myphreds)

In [ ]:
### Check encoding of phred scores
myphreds %>% encoding

In [ ]:
ShortRead::id(myrecs)

In [ ]:
as(quality(myrecs), "matrix")


In [ ]:
### Get the sequences as character strings
as.character(myseqs)

In [ ]:
### Check for duplicate reads
myseqs %>% duplicated

In [ ]:
### Get Unique reads
myseqs %>% unique

In [ ]:
### Reverse reads
myseqs %>% reverse

In [ ]:
### complement reads
myseqs %>% complement

In [ ]:
### Reverse complement reads
myseqs %>% reverseComplement

In [ ]:
### letter frequency counting
alphabetFrequency(myseqs)
letterFrequency(myseqs, c("A","T"))

## QC Report

In [ ]:
list.files(myfqdir,pattern = "1[0-1].*RZ.*.fastq.gz",full.names = TRUE) %>%
    qa(BPPARAM = MulticoreParam(workers=1)) %>%
    report(dest=file.path(outdir, "rz_10_11"))

In [ ]:
myfqfile %>%
    qa(BPPARAM = MulticoreParam(workers=1)) %>%
    report(dest=file.path(outdir, "35_MA_L002"))

## Search for subsequences

In [ ]:
### Some pattern matching
mypattern <- "AGTTGG"
matchPattern(mypattern, myseqs, max.mismatch=0)
matchPattern(mypattern, myseqs, max.mismatch=1)

In [ ]:
### Count occurences
countPattern(mypattern, myseq, max.mismatch=0) 
countPattern(mypattern, myseq, max.mismatch=1)

In [ ]:
### Alignment
pairwiseAlignment(mypattern, myseq)

## Streaming and Sampling

### Streaming
Stream a FASTQ rather than loading the whole thing into memory.

In [ ]:
mystream <- FastqStreamer(myfqfile, 5)
mystream

In [ ]:
### The first yield will retrieve records 1 through 5
cur_reads = yield(mystream)
cur_reads

In [ ]:
ShortRead::id(cur_reads)

In [ ]:
### The second yield will retrieve records 6 through 10
cur_reads = yield(mystream)
ShortRead::id(cur_reads)

In [ ]:
mystream

In [ ]:
### Be sure to close the stream
close(mystream)

### Double Check

In [ ]:
mystream <- FastqStreamer(myfqfile, 5)
cur_reads = yield(mystream)
ShortRead::id(cur_reads)
close(mystream)

In [ ]:
mystream <- FastqStreamer(myfqfile, 5)
cur_reads = yield(mystream)
ShortRead::id(cur_reads)
close(mystream)

### Sampling
Random sampling without reading in entire file upfront.
After initializing the samplers, each `yield` call will sample `n=5` records at random from the FASTQ

In [ ]:
mysampler <- FastqSampler(myfqfile, n=5)

sample_reads = yield(mysampler)
ShortRead::id(sample_reads)

close(mysampler)

In [ ]:
mysampler <- FastqSampler(myfqfile, n=5)

sample_reads = yield(mysampler)
ShortRead::id(sample_reads)

close(mysampler)

## The End

In [ ]:
sessionInfo()
q(save = "no")